#### Process raw words

In [15]:
import json
import copy

In [1]:
# with open(r'..\storage\stopWords.txt', 'r') as reader:
#      splitted_text = reader.read().split(' ')
#      proc_text = sorted(splitted_text)

# with open(r'..\storage\stop_words.txt', 'w') as writer:
#      for word in proc_text:
#        writer.write(word+'\n')

FileNotFoundError: [Errno 2] No such file or directory: '..\\storage\\stopWords.txt'

In [6]:
### Convert to tries

In [17]:
SENT_VALUE_KEY = '___'
TRIE_JSON_FILE = r'..\storage\compressed_trie.json'

In [20]:
# read postive words list from file
pos_words = []
with open(r'..\storage\pos_words.txt', 'r') as reader:
    pos_words = sorted(reader.read().split('\n'))
print(len(pos_words))

# read negative words list from file
neg_words = []
with open(r'..\storage\neg_words.txt', 'r') as reader:
    neg_words = sorted(reader.read().split('\n'))
print(len(neg_words))

# TODO: read stop words list from file
stop_words = []
with open(r'..\storage\stop_words.txt', 'r') as reader:
    stop_words = sorted(reader.read().split('\n'))
print(len(stop_words))

1419
4548
759


In [21]:
# create input variable for build_tries()
pos_values = [1] * len(pos_words)
pos_words_values = list(zip(pos_words, pos_values)) # [('a reason for being', 1), ('able', 1), ()]
print(pos_words_values[0])

neg_values = [-1] * len(neg_words)
neg_words_values = list(zip(neg_words, neg_values))
print(neg_words_values[0])

# TODO: create input variable for stop words
stop_values = [-1000] * len(stop_words)
stop_words_values = list(zip(stop_words, stop_values))
print(stop_words_values[0])

words_values = stop_words_values + pos_words_values + neg_words_values

('a reason for being', 1)
('abnormal', -1)
("'ll", -1000)


In [24]:
# build uncompressed trie
def build_tries(words_values: list) -> dict:
    'Creates a basic uncompressed trie from a sorted list of words with sentiment values.'
    root = {}
    for (word, value) in words_values:
        node = root
        word_len = len(word)
        for i, char in enumerate(word):
            # create a new child if no char found
            if char not in node:
                node[char] = {}
            
            # put a key in child indicate end of a complete word
            if i == word_len - 1:
                node[char][SENT_VALUE_KEY] = value
            node = node[char]
    return root

# create uncompressed trie
newTrie = build_tries(words_values)
print('\n'.join(json.dumps(newTrie, indent=4).split('\n')[:30]))

# with open(TRIE_JSON_FILE, 'w') as outfile:
#     json.dump(newTrie, outfile)

{
    "'": {
        "l": {
            "l": {
                "___": -1000
            }
        },
        "v": {
            "e": {
                "___": -1000
            }
        }
    },
    "a": {
        "___": -1000,
        "'": {
            "s": {
                "___": -1000
            }
        },
        "b": {
            "l": {
                "e": {
                    "___": 1
                }
            },
            "o": {
                "u": {
                    "t": {
                        "___": -1000


In [25]:
# compress the tries
def rec_search(key: str, trie: dict) -> (str, dict):
    # base case
    # end of last character of complete word
    if type(trie) is int:
        return key[:-len(SENT_VALUE_KEY)], {SENT_VALUE_KEY:trie}

    children_size = len(trie)
    newChildren = {}

    if children_size == 1:
        for childKey, childTrie in trie.items():
            prefixKey = key+childKey
            newKey, newChild = rec_search(prefixKey, childTrie)

            # special case
            # if the root tree only has one children
            if key == '':
                return '', {newKey:newChild}

            return newKey, newChild
    else:
        for childKey, childTrie in trie.items():
            if childKey == SENT_VALUE_KEY:
                newChildren[childKey] = childTrie
            else:
                prefixKey = ''+childKey
                newKey, newChild = rec_search(prefixKey, childTrie)
                newChildren[newKey] = newChild

    return key, newChildren

def compress_trie(trie: dict) -> dict:
    trie = copy.deepcopy(trie)
    _, compressedTrie = rec_search('', trie)
    return compressedTrie

# compress the trie
cTrie = compress_trie(newTrie)
print('\n'.join(json.dumps(cTrie, indent=4).split('\n')[:30]))

# write to a file
with open(TRIE_JSON_FILE, 'w') as outfile:
    json.dump(cTrie, outfile)

{
    "'": {
        "ll": {
            "___": -1000
        },
        "ve": {
            "___": -1000
        }
    },
    "a": {
        "___": -1000,
        "'s": {
            "___": -1000
        },
        "b": {
            "le": {
                "___": 1
            },
            "o": {
                "u": {
                    "t": {
                        "___": -1000
                    },
                    "nd": {
                        "___": 1,
                        "ing": {
                            "___": 1
                        },
                        "s": {
                            "___": 1


In [28]:
# test the compressed trie
def search(trie: dict, word: str) -> int:
    '''
    Helper method to search compressed trie
    '''
    word_length = len(word)
    offset = 0
    lastIndex = 0

    for i in range(word_length):
        key = word[offset:i+1]
        value = trie.get(key)

        if value is not None:
            trie = value
            offset += len(key)
            lastIndex = i

    result = trie.get(SENT_VALUE_KEY)
    if result is None or lastIndex != word_length - 1:
        return 0
    return result

# test the results 
for (word, value) in words_values:
    sent_value = search(cTrie, word)
    if(value != sent_value):
        print(word, ':', sent_value)



able : 1
allow : 1
appreciate : 1
appropriate : 1
awfully : -1
best : 1
better : 1
beyond : 1
certain : 1
do : 1
enough : 1
give : 1
giving : 1
hello : 1
help : 1
inner : 1
just : 1
like : 1
many : 1
miss : -1
more : 1
new : 1
ok : 1
on : 1
please : 1
poorly : -1
proud : 1
sorry : -1
still : 1
thank : 1
unfortunately : -1
unlikely : -1
up : 1
useful : 1
value : 1
very : 1
welcome : 1
well : 1
whole : 1
will : 1
willing : 1
wonder : 1
yes : 1
audacity : -1
blinding : -1
dope : -1
emphatic : -1
giddy : -1
incomparable : -1
joke : -1
jumpy : -1
unbelievable : -1
vulnerable : -1
zealous : -1
